In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [ ]:
nbits = 3

In [ ]:
# the variable quantum inputs
sel = QuantumRegister(1, 'sel')
p = QuantumRegister(nbits, 'p')
q = QuantumRegister(nbits, 'q')
cin = QuantumRegister(nbits, 'cin')
# plus the zero-initialized quantum input
o = QuantumRegister(1, 'o')

In [ ]:
overflow = ClassicalRegister(1, "v")
oprime = ClassicalRegister(nbits, "o'")

circ = QuantumCircuit(sel, p, q, cin, o, oprime)

In [ ]:
select = input('Perform subtraction [Y/n]: ')
mode = ''
match select:
    case 'n' | 'no' | 'No' | 'nO':
        print('Mode: Addition')
        mode = 'addition'
    case _:
        print('Mode: Subtraction')
        mode = 'subtraction'
        circ.x(sel)

In [ ]:
ip = input('Input a 3-bit number (0 <= x < 8): ')
for i, d in zip(range(nbits), reversed(bin(int(ip)).removeprefix('0b'))):
    match d:
        case '1':
            circ.x(p[i])
        case _:
            pass

    
iq = input('Input a 3-bit number (0 <= x < 8): ')
for i, d in zip(range(nbits), reversed(bin(int(iq)).removeprefix('0b'))):
    match d:
        case '1':
            circ.x(q[i])
        case _:
            pass

In [ ]:
# if sel is set, then invert one of the input registers
# we invert the |p> input register
for i in range(nbits):
    circ.cx(sel, q[i])
circ.cx(sel, cin[0])
circ.barrier()

In [ ]:
# then we apply the same gates 
# just like the full adder circuit
for i in range(nbits):
    circ.ccx(p[i], q[i], o)
    circ.cx(p[i], q[i])
    circ.ccx(q[i], cin[i], o)
    circ.cx(q[i], cin[i])
    circ.cx(p[i], q[i])
    if i+1 < nbits:
        circ.swap(o, cin[i+1])
    circ.barrier()

In [ ]:
circ.measure(cin, oprime)

In [ ]:
from qiskit.visualization import circuit_drawer
circuit_drawer(circ, output='mpl', fold=-1)

In [ ]:
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

In [ ]:
backend = Aer.get_backend('qasm_simulator')
result = backend.run(circ).result()

In [ ]:
res = list(result.get_counts().keys())[0]
op = '-'
if mode == 'addition':
    op = '+'
print(f'{bin(int(ip))} {op} {bin(int(iq))} = 0b{res}')
plot_histogram(result.get_counts())
plot_histogram(result.get_counts())